In [2]:
import gradio as gr
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input

# --- Load model ---
model = load_model("/content/drive/MyDrive/graduation project/models/OCT_InceptionV3.h5")
class_names = ["AMD", "DME", "ERM", "NO"]
last_conv_layer_name = 'conv2d_93'  # Adjust based on your model

# --- Preprocess image ---
def preprocess_image_gr(img, target_size=(299, 299)):
    img = img.resize(target_size)
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    return img_array

# --- Grad-CAM function ---
def generate_gradcam(model, img_array, class_index, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(tf.expand_dims(img_array, axis=0))
        loss = predictions[:, class_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0) / tf.math.reduce_max(heatmap + 1e-10)
    heatmap = cv2.resize(heatmap.numpy(), (299, 299))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    return heatmap

# --- Bar chart for all class probabilities ---
def plot_probabilities(probs):
    fig, ax = plt.subplots()
    ax.bar(class_names, probs, color='royalblue')
    ax.set_ylim(0, 1)
    ax.set_ylabel('Probability')
    ax.set_title('Prediction Confidence')
    plt.tight_layout()
    return fig

# --- Main prediction function ---
def predict(img):
    img_array = preprocess_image_gr(img)
    preds = model.predict(np.expand_dims(img_array, axis=0))[0]
    class_idx = np.argmax(preds)
    class_label = class_names[class_idx]
    confidence = preds[class_idx] * 100

    # Generate Grad-CAM
    gradcam = generate_gradcam(model, img_array, class_idx, last_conv_layer_name)

    # Convert Grad-CAM for display (overlay it on original image)
    img_original = np.array(img.resize((299, 299)))  # original image
    gradcam_resized = cv2.cvtColor(gradcam, cv2.COLOR_BGR2RGB)
    superimposed_img = cv2.addWeighted(img_original.astype(np.uint8), 0.6, gradcam_resized, 0.4, 0)

    # Bar chart
    prob_plot = plot_probabilities(preds)

    return f"{class_label} ({confidence:.2f}%)", superimposed_img, prob_plot

# --- Gradio Interface ---
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Label(label="Predicted Class with Confidence"),
        gr.Image(label="Grad-CAM Visualization"),
        gr.Plot(label="Prediction Probabilities")
    ],
    title="Retinal Disease Classifier (OCT Images)",
    description="Upload an OCT image to predict the disease class, see Grad-CAM, and view prediction probabilities."
)

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3871c6d19d65ca2b02.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
